# Fx Forward Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [21]:
import refinitiv.dataplatform as rdp
import pandas as pd

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
#IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DESKTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Computing the Mark-to-Market of an Fx Forward instrument

This sample compute the market value in each currency of an existing forward. it also provide additional information like:
- the sport rate ("FxSpot") and the cross outright and swap points at the maturity of the forward ("FxOutrightCcy1Ccy2", "FxSwapsCcy1Ccy2")
- the delta in each currency ("DeltaAmountInDealCcy", "DeltaAmountInContraCcy")

In [22]:
request_body={
    "fields" : ["InstrumentTag", "StartDate","FxSpot", "FxOutrightCcy1Ccy2", "FxSwapsCcy1Ccy2",\
                "MarketValueInDealCcy", "MarketValueInContraCcy", "DeltaAmountInDealCcy", "DeltaAmountInContraCcy",\
                "ErrorCode", "ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType" : "FxCross",
            "instrumentDefinition" : {
                "instrumentTag" : "1",
                "fxCrossType" : "FxForward",
                "fxCrossCode" : "EURUSD",
                "legs" : [
                    {
                        "endDate" : "2020-03-09T00:00:00Z",
                        "startDate" : "2020-03-06T00:00:00Z",
                        "legTag" : "L:0-R:0",
                        "fxLegType" : "FxForward",
                        "dealCcy" : "EUR",
                        "dealCcyBuySell" : "buy",
                        "dealAmount" : 2000000,
                        "contraAmount" : 2224982
                    }]
            },
            
            "pricingParameters" : {
                "marketDataDate" : "2020-03-06T00:00:00Z",
                "valuationDate" : "2020-03-06T00:00:00Z"
            }
        }],
    
    "outputs" : ["Data","Headers"]
}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,StartDate,FxSpot,FxOutrightCcy1Ccy2,FxSwapsCcy1Ccy2,MarketValueInDealCcy,MarketValueInContraCcy,DeltaAmountInDealCcy,DeltaAmountInContraCcy,ErrorCode,ErrorMessage
0,1,2020-03-06T00:00:00Z,1.1286,1.128382,1.61,28167.167052,31779.020724,1.971809e+06,1.999974e+06,,
